# Lesson 4: Building a Multi-Document Agent

## Setup

In [8]:
import re
import os
import logging
import pandas as pd
import openai
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
pd.set_option('display.max_colwidth', None)

In [9]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
from utils_rag import get_doc_tools
from pathlib import Path

In [11]:
from llama_index.llms.openai import OpenAI
openai.api_key = OPENAI_API_KEY
llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)

In [12]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

In [13]:
logging.basicConfig(level=logging.ERROR)

# Set up logging for OpenAI
openai.logging = logging.getLogger("openai")
openai.logging.setLevel(logging.ERROR)

## 2. Setup an agent over articles

### Download 11 ICLR papers

In [14]:
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=LzPWWPAdY4",
    "https://openreview.net/pdf?id=VTF8yNQM66",
    "https://openreview.net/pdf?id=hSyW5go0v8",
    "https://openreview.net/pdf?id=9WD9KwssyT",
    "https://openreview.net/pdf?id=yV6fD7LYkF",
    "https://openreview.net/pdf?id=hnrB5YHoYu",
    "https://openreview.net/pdf?id=WbWtOYIzIK",
    "https://openreview.net/pdf?id=c5pwL0Soay",
    "https://openreview.net/pdf?id=TpD2aG1h0D"
]

papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "loftq.pdf",
    "swebench.pdf",
    "selfrag.pdf",
    "zipformer.pdf",
    "values.pdf",
    "finetune_fair_diffusion.pdf",
    "knowledge_card.pdf",
    "metra.pdf",
    "vr_mcl.pdf"
]

In [15]:
def list_files_recursively(directory):
    paths=[]
    for root, dirs, files in os.walk(directory):
        for file in files:
            paths.append(os.path.join(root, file))
    return paths

# Example usage
papers=list_files_recursively('support_articles/')
len(papers)

130

In [16]:
# for url, paper in zip(urls, papers):
#      !wget "{url}" -O "{paper}"

To download these papers, below is the needed code:


    #for url, paper in zip(urls, papers):
         #!wget "{url}" -O "{paper}"
    
    
**Note**: The pdf files are included with this lesson. To access these papers, go to the `File` menu and select`Open...`.

In [17]:
def sanitize_names(name):
    name=re.sub('[^0-9a-zA-Z]+', '_', name)
    return name

In [18]:
paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, sanitize_names(Path(paper).stem))
    if len(vector_tool.metadata.name)>64:
        resp=llm.complete(f'please make this function name shorter so that it would have only up to 60 or less characters {vector_tool.metadata.name}')
        vector_tool.metadata.name=resp.text[:64]
    if len(summary_tool.metadata.name)>64:
        resp=llm.complete(f'please make this function name shorter so that it would have only up to 60 or less characters {summary_tool.metadata.name}')
        summary_tool.metadata.name=resp.text[:64]
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: support_articles/FS Text\Administrator tools.docx
Getting tools for paper: support_articles/FS Text\Audio segments.docx
Getting tools for paper: support_articles/FS Text\Classification and topic model data processing differences.docx
Getting tools for paper: support_articles/FS Text\FS Text application overview.docx
Getting tools for paper: support_articles/FS Text\FS TEXT APPLICATION тАУ USER TYPES, ADMINISTRATING AND CREATING.docx
Getting tools for paper: support_articles/FS Text\FStext_articles_structure.docx
Getting tools for paper: support_articles/FS Text\How to add models into production environment.docx
Getting tools for paper: support_articles/FS Text\How to assess a classification model.docx
Getting tools for paper: support_articles/FS Text\How to change models attached to production environment.docx
Getting tools for paper: support_articles/FS Text\How to create a classification model.docx
Getting tools for paper: support_articles/FS Text\How to crea

Getting tools for paper: support_articles/Topic Search and search results\Event detail view.docx
Getting tools for paper: support_articles/Topic Search and search results\Event quick view.docx
Getting tools for paper: support_articles/Topic Search and search results\Examples of search queries.docx
Getting tools for paper: support_articles/Topic Search and search results\How to search.docx
Getting tools for paper: support_articles/Topic Search and search results\How to view search results.docx
Getting tools for paper: support_articles/Topic Search and search results\Listening to a call.docx
Getting tools for paper: support_articles/Topic Search and search results\Normalized time тАУ searching from a particular part in a call or chat.docx
Getting tools for paper: support_articles/Topic Search and search results\Searching using a phone number.docx
Getting tools for paper: support_articles/Topic Search and search results\Sharing a conversation with a colleague.docx
Getting tools for paper:

### Extend the Agent with Tool Retrieval

In [19]:
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [20]:
len(all_tools)

260

In [21]:
for tool in all_tools:
    print(len(tool.metadata.name))

31
32
26
27
39
41
40
41
37
46
37
38
57
58
48
49
37
32
48
49
41
42
39
40
54
55
34
47
30
31
25
26
43
44
36
37
48
49
61
62
34
35
54
55
52
53
31
32
51
52
40
41
58
59
42
43
48
49
25
26
42
43
36
37
31
32
28
33
47
48
51
52
35
36
47
48
37
38
60
61
22
23
48
49
22
23
47
37
36
56
45
46
60
61
35
36
60
61
46
47
54
55
37
38
36
37
43
44
44
45
37
38
36
37
32
33
38
39
37
38
64
40
51
38
39
40
43
44
53
54
50
51
54
55
50
51
42
43
47
59
32
33
41
42
43
44
45
46
46
47
43
50
41
42
58
59
48
49
47
48
53
54
43
44
57
58
29
30
28
29
38
39
25
26
38
39
31
32
60
61
42
43
51
52
45
46
47
48
29
18
25
26
44
39
35
36
43
44
54
55
47
48
59
60
61
62
64
64
46
47
33
34
36
37
35
31
64
50
63
64
38
26
60
64
64
33
61
63
44
64
34
35
35
36
39
40
52
53
39
42
27
28
49
50
40
41
24
25
20
21
33
34
41
42
30
31
38
39
25
26


In [22]:
all_tools[5].metadata.name

'summary_tool_Class_topic_processing_diffs'

In [23]:
# define an "object" index and retriever over these tools
obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
    verbose=True
)

In [24]:
# obj_index.persist(persist_dir='object_index')

In [25]:
# ObjectIndex.from_persist_dir(persist_dir='object_index')

In [26]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3, verbose=True)

In [27]:
tools = obj_retriever.retrieve(
    "how to build a wordcloud?"
)

In [28]:
tools

In [47]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    system_prompt=""" \
You are an agent designed to answer queries over a set of given articles.
Please always use the tools provided to answer a question. Do not rely on prior knowledge. Say if topic is not mentioned in the articles\

""",
    verbose=True
)
agent = AgentRunner(agent_worker, verbose=True)

## use agent

In [48]:
response = agent.query(
    "how to create rules?"
)
print(str(response))

> Running step dd84c66f-e7fe-43d9-b6ac-4bf273b07d50. Step input: how to create rules?
Added user message to memory: how to create rules?
=== Calling Function ===
Calling function: summary_tool_ADMIN_How_to_create_Rules_and_tie_them_to_Actions with args: {"input": "how to create rules"}
=== Function Output ===
To create rules, first, go to the Rules view in the Tasks menu and click on the plus sign to set up a new rule. Fill in the Rule Settings with the name and choose the Action to tie the rule to. You can set the "Continue" field to yes or no, indicating whether other rules should be performed if the conditions are met. Add filters to set up specific criteria for the rule, such as using filters like "text contains" to query specific fields like overall event text or actor. Apply filters as needed to ensure the rule captures the desired events. Regularly review and manage the rules list to remove any unnecessary rules and actions for clarity and efficiency.
> Running step 5dccb040-fa9

In [49]:
response = agent.query(
    "what I need to train topic model?"
)

> Running step c43c7373-f66e-4217-bfce-012520c7cc0e. Step input: what I need to train topic model?
Added user message to memory: what I need to train topic model?
=== Calling Function ===
Calling function: summary_tool_What_aspects_to_think_about_before_model_training with args: {"input": "What do I need to train a topic model?"}
=== Function Output ===
To train a topic model, you need texts as input. It is important to have at least hundreds of samples of data, with more data being preferable for better results. If you do not have any labelled data, starting with a topic model is recommended. The topic model requires a sufficient amount of texts to work effectively, ideally more than 1000 texts for more granular topic discovery.
> Running step 3f23f3f1-64f0-449c-8b17-7d78eabd148b. Step input: None
=== LLM Response ===
To train a topic model, you need texts as input. It is important to have at least hundreds of samples of data, with more data being preferable for better results. If you

In [50]:
response = agent.query(
    "How can I change the time period from days to weeks when making charts?"
)

> Running step e58ba026-0559-4819-b28f-6292db25cc6a. Step input: How can I change the time period from days to weeks when making charts?
Added user message to memory: How can I change the time period from days to weeks when making charts?
=== Calling Function ===
Calling function: summary_tool_How_to_apply_filters_during_chart_creation with args: {"input": "Change time period from days to weeks when making charts"}
=== Function Output ===
To change the time period from days to weeks when making charts, you can utilize the filters available during chart creation. By specifying the desired time frame in terms of weeks instead of days, you can adjust the results displayed in the chart accordingly. This flexibility allows you to customize the chart output based on your specific requirements, providing a more tailored visualization of the data.
> Running step 77675b51-483a-4bc4-9049-8f0427556916. Step input: None
=== LLM Response ===
To change the time period from days to weeks when making 

In [51]:
response = agent.query(
    "What is new in version 1.11?"
)

> Running step a02dcb75-2e87-4771-851e-c8353681998a. Step input: What is new in version 1.11?
Added user message to memory: What is new in version 1.11?
=== Calling Function ===
Calling function: summary_tool_RELEASE_NOTES_FOR_VERSION_1_11 with args: {"input": "What is new in version 1.11?"}
=== Function Output ===
In version 1.11, two new functions have been introduced. The first one is consent-based conversation analytics, which allows individuals to choose whether their data is used for analysis purposes. The second new function is time-limited data access, which enables setting time-based limits for accessing data in the Leadstream application.
> Running step 8faa320c-2403-475d-a5c2-938bb42dcfec. Step input: None
=== LLM Response ===
In version 1.11, two new functions have been introduced: consent-based conversation analytics and time-limited data access.


In [52]:
response = agent.query(
    "Can I use LLMs in the platform?"
)

> Running step fa2e4746-4c0c-4a81-99bc-cec003dc3f79. Step input: Can I use LLMs in the platform?
Added user message to memory: Can I use LLMs in the platform?
=== LLM Response ===
I will check the articles to find information about using LLMs in the platform.
=== Calling Function ===
Calling function: summary_tool_Language_detection with args: {"input": "LLMs in the platform"}
=== Function Output ===
The platform does not mention LLMs in the context provided.
> Running step f167144d-99cc-49e9-a627-365df89287ed. Step input: None
=== LLM Response ===
The platform does not mention the use of LLMs in the context provided.


In [53]:
response = agent.query(
    "Can I fly to the moon with the platform?"
)

> Running step 01565f15-bae8-4d95-8d48-677d24b6194b. Step input: Can I fly to the moon with the platform?
Added user message to memory: Can I fly to the moon with the platform?
=== LLM Response ===
I'm not sure what you mean by "the platform." Could you please provide more context or clarify your question so I can better assist you?


In [54]:
response = agent.query(
    "Can I edit transcript in the platform?"
)

> Running step 10ac9bed-3c9c-45ca-b2ce-c893d31084ec. Step input: Can I edit transcript in the platform?
Added user message to memory: Can I edit transcript in the platform?
=== Calling Function ===
Calling function: summary_tool_ADMIN_How_to_edit_the_Conversations_views with args: {"input": "Can I edit transcript in the platform?"}
=== Function Output ===
Yes, you can edit the transcript in the platform.
> Running step a40dc606-a05f-497e-8e45-02926f33186e. Step input: None
=== LLM Response ===
Yes, you can edit the transcript in the platform.


In [55]:
response = agent.query(
    "please give me a detailed guide how to edit transcript"
)

> Running step 95c8eab6-4b5d-4ea5-9419-c7fb74c3b0f2. Step input: please give me a detailed guide how to edit transcript
Added user message to memory: please give me a detailed guide how to edit transcript
=== Calling Function ===
Calling function: summary_tool_ADMIN_How_to_edit_the_Conversations_views with args: {"input": "detailed guide how to edit transcript"}
=== Function Output ===
The article provides a detailed guide on how an Administrator user can edit the Dashboard view in the Conversations menu. It explains that the Administrator can adjust the Dashboard view based on the company's needs by accessing the Story settings cogwheel in the top right corner of the page. By selecting "Edit this template," the Administrator can then make changes to all charts within the Dashboard, add new ones, delete them, add comments, and more. It emphasizes the importance of considering user permissions when making changes and suggests creating separate Storyboards if needed. The article also men

## try with trulens eval

In [56]:
import nest_asyncio

nest_asyncio.apply()

In [57]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI(api_key=OPENAI_API_KEY)

#### define metrics

In [58]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [59]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [60]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\RistoHinno\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## load example questions

In [61]:
eval_questions = []
with open('data/eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [62]:
answer_relevances=[]
groundednesses=[]
context_relevances=[]
contexts=[]
answers=[]

for question in eval_questions:
    response = agent.query(question)
    answers.append(response.response)
    
    answer_relevances.append(f_qa_relevance(question, response.response))
    
    context=' '.join([t.text for t in response.source_nodes])
       
    contexts.append(context)
    context_relevances.append(f_qs_relevance(question, context))
    
    if context=='':
        groundednesses.append(({'statement_0': 0},
                              {'reasons': 'no_text'}))
    else:
        groundednesses.append(f_groundedness(context, response.response))

> Running step adbf8d3d-aede-4159-a538-c646fc19949f. Step input: how can I make topic model?
Added user message to memory: how can I make topic model?
=== Calling Function ===
Calling function: summary_tool_How_to_create_a_topic_model with args: {"input": "how can I make topic model?"}
=== Function Output ===
To create a topic model using the FS Text application, you can follow these steps:

1. Log in to the FS Text application and choose the Model builder menu item.
2. In the Model builder view, click on the New model option in the left-hand menu.
3. Fill in the form for your new model with a descriptive name, description, business purpose, and language. Choose Topic model as your Model type and press Create model.
4. Upload your data using an Excel file in the Model and data view. Ensure all data is included in one Excel file.
5. Review the data to validate if it was uploaded correctly in the Review data view.
6. Add words to be removed in the Cleaners tab, such as stopwords, in a sp

Groundedness per statement in source:   0%|          | 0/24 [00:00<?, ?it/s]

> Running step c01d5827-57ff-4755-80a7-8ba4fc4aa486. Step input: how much data is needed for topic model?
Added user message to memory: how much data is needed for topic model?
=== Calling Function ===
Calling function: summary_tool_How_to_create_a_topic_model with args: {"input": "how much data is needed for topic model"}
=== Function Output ===
The data needed for creating a topic model can vary depending on the specific use case and the complexity of the topics being analyzed. Typically, a topic model can be created with a few hundred to a few thousand texts as input data. It is important to have a sufficient amount of data to ensure that the model can identify meaningful topics and patterns within the text.
> Running step 61bbaad5-74bf-437b-9340-6d0873a5d81c. Step input: None
=== LLM Response ===
The data needed for creating a topic model can vary, but typically a few hundred to a few thousand texts are required as input data.


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

> Running step 9921c84d-697b-42d8-a5fc-61b96f1af47e. Step input: can LLMs be used for topic models?
Added user message to memory: can LLMs be used for topic models?
=== Calling Function ===
Calling function: summary_tool_How_to_create_a_topic_model with args: {"input": "LLMs"}
=== Function Output ===
LLMs are not directly mentioned in the provided context information.
> Running step 58b977ac-a6b7-48ac-a072-0f547907d8d1. Step input: None
=== LLM Response ===
LLMs (Large Language Models) are not directly mentioned in the context of creating topic models. If you need more specific information on how LLMs can be used for topic models, please provide additional details or context for a more accurate response.


...

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

> Running step 3105295c-1c35-4b71-941e-aabfc9f2aac7. Step input: where should I start if I am total beginner in using the application?
Added user message to memory: where should I start if I am total beginner in using the application?
=== Calling Function ===
Calling function: summary_tool_How_to_get_started_with_analysing_customer_feedback with args: {"input": "total beginner in using the application"}
=== Function Output ===
The article provides a step-by-step guide on how to get started with analyzing customer feedback using the Leadstream application. It explains the importance of understanding the data available, creating an overview, setting goals or questions, validating and measuring hypotheses, and monitoring changes over time. By following these steps, a total beginner in using the application can gradually learn how to effectively analyze customer feedback and derive insights to improve customer experience.
> Running step ae5dd006-e432-4307-a9fd-e16b2412454f. Step input: Non

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

> Running step 5a610e2a-efee-4ae7-b548-25b212ba5c75. Step input: can I download data into excel file?
Added user message to memory: can I download data into excel file?
=== LLM Response ===
To assist you with downloading data into an Excel file, I will first need to understand the context or source of the data you are referring to. Could you please provide more details about the data source or the specific data you would like to download into an Excel file?
> Running step 88949921-b431-4ec0-baa0-8c81288d98f7. Step input: anonymization is not working correctly, some names are not removed. What should I do?
Added user message to memory: anonymization is not working correctly, some names are not removed. What should I do?
=== Calling Function ===
Calling function: summary_tool_Anonymisation_of_conversations with args: {"input": "anonymization is not working correctly, some names are not removed. What should I do?"}
=== Function Output ===
To address the issue where some names are not bein

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

## fromat results

In [63]:
df_ar=pd.DataFrame(answer_relevances)
df_ar.columns=['answer_relevance_score','relevance_reason']
# df_ar

In [64]:
def get_grounded_scores(scores, use_median=False):
    scores_all=[]
    for key, value in scores.items():
        scores_all.append(value)
    if use_median:
        return np.median(scores_all)
    return np.mean(scores_all)

In [65]:
df_=pd.DataFrame(groundednesses)
df_.columns=['ground_scores','ground_reason']
df_['ground_score']=df_.ground_scores.apply(lambda x: get_grounded_scores(x))

df_results=pd.concat([df_ar, df_], axis=1)
df_results['context_relevance_score']=context_relevances
df_results['question']=eval_questions
df_results['answer']=answers

In [66]:
df_results[['answer_relevance_score','ground_score', 'context_relevance_score', 'question','answer']]

,answer_relevance_score,ground_score,context_relevance_score,question,answer
0,0.9,1.00,0.8,how can I make topic model?,"assistant: To create a topic model using the FS Text application, you can follow these steps:\n\n1. Log in to the FS Text application and choose the Model builder menu item.\n2. In the Model builder view, click on the New model option in the left-hand menu.\n3. Fill in the form for your new model with a descriptive name, description, business purpose, and language. Choose Topic model as your Model type and press Create model.\n4. Upload your data using an Excel file in the Model and data view. Ensure all data is included in one Excel file.\n5. Review the data to validate if it was uploaded correctly in the Review data view.\n6. Add words to be removed in the Cleaners tab, such as stopwords, in a specific format.\n7. Clean the data in the Clean data area, which involves processes like lowercasing text, removing punctuation, and lemmatization.\n8. Generate ngrams in the Ngrams tab to grasp common terms in the text.\n9. Start the model training process by providing the number of topics and clicking on the Train model button in the Train model view.\n10. Review the training experiments and adjust the number of topics if needed in the Experiments view.\n11. Test the trained model by writing/copying a text in the Predict view.\n12. Finish up by iterating through cleaning and training rounds, ensuring the topics make sense and experimenting with different numbers of topics until you achieve informative results."
1,0.9,0.70,0.4,how much data is needed for topic model?,"assistant: The data needed for creating a topic model can vary, but typically a few hundred to a few thousand texts are required as input data."
2,0.2,0.00,0.4,can LLMs be used for topic models?,"assistant: LLMs (Large Language Models) are not directly mentioned in the context of creating topic models. If you need more specific information on how LLMs can be used for topic models, please provide additional details or context for a more accurate response."
3,1.0,1.00,0.7,what are the steps for training topic model?,"assistant: Training a topic model involves several steps. Here is an overview of the process:\n\n1. Log in to the FS Text application and access the Model builder menu.\n2. Create a new model by providing details like name, description, business purpose, and language.\n3. Choose ""Topic model"" as the Model type and create the model.\n4. Upload data in the Model and data view using an Excel file.\n5. Review the data in the Review data view to validate the correct format and content.\n6. Clean the data by removing stopwords and other unnecessary words in the Cleaners tab.\n7. Generate ngrams for common terms in the text in the Ngrams tab.\n8. Train the model by specifying the number of topics and initiating the training process in the Train model view.\n9. Monitor the training progress in the Logs under Training logs.\n10. Review the results in the Experiments view to evaluate topics, adjust as needed, and name them accordingly.\n11. Test the trained model in the Predict view by inputting text to predict its topic.\n\nThe training process may involve multiple rounds of cleaning and training to refine the model for accurate topic classification. It is crucial to ensure that the topics generated make sense and are distinct."
4,1.0,1.00,0.8,What kind of analysis can I make in the platform?,"assistant: You can conduct various types of analysis on the platform, including real-time conversation analysis, short-term monitoring and evaluation, medium-term trend tracking, long-term performance reviews, competitor analysis, customer satisfaction trend analysis, product/service feedback analysis, conversation topic analysis, seasonal trend identification, customer lifecycle analysis, long-term strategy development, customer loyalty analysis, predictive analytics, and reclamation and complaint handling."
5,1.0,0.00,0.0,how to find unwanted calls?,"assis

In [67]:
df_results[['answer_relevance_score','ground_score', 'context_relevance_score', 'question','answer']].to_excel('rag_agent.xlsx')

C:\Users\RistoHinno\AppData\Local\Temp\ipykernel_10584\3876928364.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_results[['answer_relevance_score','ground_score', 'context_relevance_score', 'question','answer']].to_excel('rag_agent.xlsx')


## try to correct the answer

In [45]:
#take answer that has low groundedness
prompt="I asked a question 'can LLMs be used for topic models?' and you gave this answer which was not gorunded based on the documents. please give correct answer which is based only on documents and not on previous knowledge. if documents don't mention this topic, say so. Old answer: 'LLMs (Large Language Models) can be used for topic modeling tasks. These models, like BERT (Bidirectional Encoder Representations from Transformers) and GPT (Generative Pre-trained Transformer), have been applied to various natural language processing tasks, including topic modeling. LLMs can be fine-tuned on specific datasets to perform topic modeling tasks effectively. By leveraging the contextual understanding and representation capabilities of LLMs, researchers and practitioners can create topic models that capture the underlying themes and topics present in a corpus of text data.'"

In [46]:
response = agent.query(prompt)

> Running step a71f405f-f481-4b82-b8f3-481e57c8f4a2. Step input: I asked a question 'can LLMs be used for topic models?' and you gave this answer which was not gorunded based on the documents. please give correct answer which is based only on documents and not on previous knowledge. if documents don't mention this topic, say so. Old answer: 'LLMs (Large Language Models) can be used for topic modeling tasks. These models, like BERT (Bidirectional Encoder Representations from Transformers) and GPT (Generative Pre-trained Transformer), have been applied to various natural language processing tasks, including topic modeling. LLMs can be fine-tuned on specific datasets to perform topic modeling tasks effectively. By leveraging the contextual understanding and representation capabilities of LLMs, researchers and practitioners can create topic models that capture the underlying themes and topics present in a corpus of text data.'
Added user message to memory: I asked a question 'can LLMs be u